In [1]:
using ParallelTemperingMonteCarlo
using BenchmarkTools
using Revise
using DelimitedFiles
using StaticArrays

In [2]:
pwd()

"/home/ghun245/ParallelTemperingMonteCarlo.jl/scripts"

The next cell just initialises the Ne13 cluster to test the saving functions and test how they are read in

In [3]:
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 300000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

32-element Vector{MCState{Float64, 13, SphericalBC{Float64}}}:
 MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[5.339202509675499, 1.7547286276557148, 0.9552945639202041], [3.8235622875401982, -4.036693324695786, 1.2586922223528678], [3.8432525502537267, -1.2169635315645375, -4.030786193502644], [1.8515104475315411, 4.369051328639683, -3.1594504692142205], [1.8196441394724319, -0.19336921007065874, 5.399104677171738], [0.6008527770958447, 5.001668105430829, 2.668542641160762], [-5.339209779512827, -1.754731061649526, -0.9552951725503948], [-0.6008496122188508, -5.001669753738395, -2.668542209586626], [-1.851508499387977, -4.369053575565342, 3.1594501498809775], [-1.8196475277773039, 0.19336884278473446, -5.399104511708638], [-3.823559204867802, 4.036697652557049, -1.2586918476896083], [-3.8432544820617607, 1.2169644600549232, 4.030785366187147], [4.394257284152319e-6, 1.4401613097476533e-6, 7.84105389618

In [4]:
MCRun.save_states(mc_params,mc_states,1)
readingfile = open("save.data", "r+")
data = readdlm(readingfile,skipstart=1)
close(readingfile)
data[13,:]

9-element Vector{Any}:
    "Boundary:"
    "SphericalBC{Float64}"
 101.06969058367278
    ""
    ""
    ""
    ""
    ""
    ""

In [5]:
function readinput(savedata)
    paramdata = savedata[1:6 ,:] #when writing this function properly, remember there is a steps line at the top.
    configdata = savedata[7:end,:]

    return paramdata,configdata
end
paramdata,configdata = readinput(data)
i = 6
println(paramdata[i,1]," dundundun ", paramdata[i,2])
for i = 1:6
    println(typeof(paramdata[i,2]))
end
println(configdata[9,1:3])


n_adjust: dundundun 100


SubString{

String}
Int64
Int64
Int64
Int64
Int64
Any

[5.339202509675499, 1.7547286276557148, 0.9552945639202041]


Reading one configuration is simple enough, but first we need to do some testing concerning the type of the input

In [6]:


function initialiseparams(paramdata)

    MC_param = MCParams(paramdata[2,2],paramdata[4,2],paramdata[5,2],mc_sample = paramdata[3,2], n_adjust = paramdata[6,2])

    return MC_param
end

function calclength(mcparams)
    infolines = 9
    poslines = mcparams.n_atoms

    lines = Int64(infolines + poslines)
    return lines 
end

function readconfig(oneconfigvec,n_atoms, potential)
    positions = []
    coord_atom = zeros(3)
    for j=1:n_atoms
        coord_atom = SVector(oneconfigvec[8+j,1] ,oneconfigvec[8+j,2] ,oneconfigvec[8+j,3] )
        push!(positions,coord_atom)
    end
    if oneconfigvec[7,2] == "SphericalBC{Float64}"
        boundarycondition = SphericalBC(radius=oneconfigvec[7,3])
    end
    counta = [oneconfigvec[5,2], oneconfigvec[5,3]]
    countv = [oneconfigvec[5,4], oneconfigvec[5,5]]
    countr = [oneconfigvec[5,6], oneconfigvec[5,7]]
    countx = [oneconfigvec[5,8], oneconfigvec[5,9]]
    
    mcstate = MCState(oneconfigvec[2,2], oneconfigvec[2,3],Config(positions,boundarycondition), potential ; max_displ=[oneconfigvec[4,2], oneconfigvec[4,3], oneconfigvec[4,4] ], count_atom=counta,count_vol=countv,count_rot=countr,count_exc=countx)

    return mcstate
end



    
line = calclength(initialiseparams(paramdata))

configdata[19,1] == configdata[19+line,1]



true

In [7]:
state_test = readconfig(configdata[1:line,:],13,pot)
readingfile1 = open("save.data", "r+")
data1 = readdlm(readingfile1)
close(readingfile1)
data1[1,5]

1

In [8]:
mc_params,mc_states,step = restart_ptmc(pot)

MethodError: MethodError: objects of type Matrix{Any} are not callable
Use square brackets [] for indexing an Array.